In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import xgboost as xgb

In [ ]:
train = pd.read_csv('../input/jane-street-market-prediction/train.csv',nrows=200000)

train = train.astype({c: np.float16 for c in train.select_dtypes(include='float64').columns}) #limit memory use
train['feature_64'] = train['feature_64'].interpolate()
train.fillna(train.mean(),inplace=True)

Assuming `feature_64` is time in half hours starting from 9.30. 

In [ ]:
train['datetime'] = pd.to_datetime(train['date'], unit='d') + pd.to_timedelta(9.5+0.5*(train['feature_64']-train['feature_64'].min()), unit='h')
train.set_index('datetime',inplace=True)

In [ ]:
features = [col for col in train.columns if 'feature' in col]

In [ ]:
#Dropping feature_0
for f in features[1:]:
    for ws in ['20s','5min']:
        train[f+f'_rolling_{ws}'] = train[f].rolling(ws).mean()
features = [col for col in train.columns if 'feature' in col]

X = train[features].values
y = (train['resp'] > 0).astype('int')

In [ ]:

clf = xgb.XGBClassifier(
    n_estimators=400,
    max_depth=7,
    eta=0.5, 
    missing=None,
    random_state=42,
    tree_method='gpu_hist',
    subsample=0.8,
    colsample_bytree=1,
    verbosity=2   
)

In [ ]:
%time clf.fit(X, y)

In [ ]:
sorted(list(zip(features,clf.feature_importances_.ravel())),key=lambda x: x[1],reverse=True)